In [1]:
#pip install voila
#pip install ipyfilechooser
import geemap
import os
import ee
import geemap
import ipywidgets as widgets
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
#conda install mamba -c conda-forge
#conda install xarray_leaflet -c conda-forge

##Calculo de Tasa de crecimiento para cultivos mediante imagenes Sentinel. 
###frolla.franco@inta.gob.ar

In [2]:

#import geemap.eefolium as geemap
Map = geemap.Map(center=[-36,-63], zoom=4)
url = 'http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Map', attribution='Google')
Map.addLayerControl()

Map

Map(center=[-36, -63], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
import os
#from ipyfilechooser import FileChooser
import Filechooser_fran as fcf
fc1 = fcf.FileChooser_fran(os.getcwd())
#fc1 = FileChooser(os.getcwd())
fc1.filter_pattern = ['*.shp']


# Print the selected path, filename, or both
fc1

FileChooser(path='C:\Users\francofrolla\Google Drive\Michael Botham', filename='', title='HTML(value='', layou…

In [4]:
import config
import ee
import json
import tempfile
import geopandas as gpd


style = {'description_width': 'initial'}

output = widgets.Output()

subirkml = widgets.Button(
    description='Agregar Lote',
    button_style='primary',
    tooltip='Click para buscar imagenes',
    style=style
)



def subirkml_clicked(b):
    
 ee.Initialize()
     
 dirpath = os.getcwd()
 ruta = fc1.selected_path + "\\" + fc1.selected_filename
 df = gpd.read_file(ruta)
 direccion = dirpath
 rutafinal = direccion+"\\"+"salida1.geojson"
 df = df.to_crs("EPSG:4326")
 df.to_file(rutafinal, driver='GeoJSON')
    
 #Genero la feature collection
 listaobjeto=ee.List([])
 f=open(rutafinal, "r")
 contents =f.read()
 geojson1 = contents.replace(", 0.0", "")
 d = json.loads(geojson1)
 for a in range(0,len(d['features'])):
  geometria = ((d['features'][a]["geometry"]["coordinates"]))
  propiedades = d["features"][a]["properties"]
  objeto = ee.Feature(ee.Geometry.Polygon(geometria[0]), propiedades)
  listaobjeto = listaobjeto.add(objeto)

 fet = ee.FeatureCollection(listaobjeto)
 config.fet = fet
 f.close()

 df['dissolvefield'] = 1
 df1 = df.dissolve(by='dissolvefield')
 df1 = df1["geometry"]
 config.df1 = df1
 rutafinal2 = direccion+"\\"+"salida2.geojson"
 df1.to_file(rutafinal2, driver='GeoJSON')

 f=open(rutafinal2, "r")
 contents =f.read()
 geojson1 = contents.replace(", 0.0", "")
  
 if 1 == 1:    
  d = json.loads(geojson1)
  geometria = (d['features'][0]["geometry"]["coordinates"])
  lote = ee.Geometry.Polygon(geometria[0])
  config.lote = lote
  Map.addLayer(lote,{}, 'Layer name')
  Map.centerObject(lote, 14)
  Map.addLayerControl()
  Map
  f.close()
 else:
  texto.value = "No se selecciono ningun KML"
  

subirkml.on_click(subirkml_clicked)
subirkml

Button(button_style='primary', description='Agregar Lote', style=ButtonStyle(), tooltip='Click para buscar ima…

In [5]:


from ipywidgets import Checkbox, HBox, VBox
from IPython.display import display, clear_output
import ipyleaflet

import json
import datetime
from datetime import date
import config

boton0 = widgets.Button(description='Buscar Imagenes',button_style='primary')
selfechas = widgets.Dropdown(
    options=config.listaid,
    value=config.listaid[0],
    description='Seleccione Fecha:',
    disabled=False,
)


out = widgets.Output()

def buscar_imagenes(b):
    start_date = date.today()
    fecha = start_date.strftime("%Y-%m-%d")
    fecha_inicio = ee.Date(fecha);


    #fecha_inicio = ee.Date(fecha);
    #fecha_inicio = ee.Date('2020-03-15')
    fecha_fin =  fecha_inicio.advance(-15, 'day')


    lote = config.lote


    loteentero = lote
    lote = lote.buffer(-35)

    coleccionfiltrada = ee.ImageCollection('COPERNICUS/S2').filterBounds(lote).filterDate(fecha_fin,fecha_inicio)

    lista = coleccionfiltrada.toList(coleccionfiltrada.size())
    imagen = ee.Image(lista.get(0))
    lista = lista.add(imagen)

    def detectar_duplicador(imagen):
        esduplicado = ee.String("")
        numero = lista.indexOf(imagen)
        imagen1 = ee.Image(lista.get(numero.add(1)))
        #Compare the image(0) in the ImageCollection with the image(1) in the List
        fecha1 = imagen.date().format("Y-M-d")
        fecha2 = imagen1.date().format("Y-M-d")
        estado = ee.Algorithms.IsEqual(fecha1,fecha2)
        esduplicado = ee.String(ee.Algorithms.If(estado,"duplicado","no duplicado"));
        imagen = imagen.set("duplicado", esduplicado)
        return imagen

    coleccionfiltrada = coleccionfiltrada.map(lambda image: detectar_duplicador(image))
    coleccionfiltrada = coleccionfiltrada.filter(ee.Filter.eq('duplicado', "no duplicado"))

    def fechas(image):
         fecha = image.select("B3").date().format("YYYY-MM-dd")
         image = image.set("fecha",fecha)
         return image

    coleccionfiltrada = coleccionfiltrada.map(lambda imagen: fechas(imagen))

    with out:
     print("Armando diccionario y descargando datos del servidor")

    fechas = coleccionfiltrada.aggregate_array("fecha")
    id = coleccionfiltrada.aggregate_array("system:index")

    test_dict = ee.Dictionary.fromLists(['fechas', 'id'], [fechas, id])
    featureCollection = ee.FeatureCollection([ee.Feature(None, test_dict)])

    link = featureCollection.getDownloadURL(filetype="CSV", selectors=None, filename=None)
    #Generamos el diccionario con los valores.
    import csv, urllib.request
    response = urllib.request.urlopen(link)
    lines = [l.decode('utf-8') for l in response.readlines()]
    reader = csv.DictReader(lines)

    #import ast
    #for row in reader:
    # x = row["fechas"]
    # fechas = x
    # x1 = row["id"]
    # id = x1
        
    
    data = list(reader)
    # Converting string to list
    fechas_cliente = data[0]["fechas"].strip('][').split(', ')
    id_cliente = data[0]["id"].strip('][').split(', ')
    
    #config.fechas_cliente = fechas_cliente
    config.id_cliente = id_cliente

    fechas_cliente = map(str, fechas_cliente)
    fechas = list(fechas_cliente)

    #id_cliente = map(str, id_cliente)
    #id = list(id_cliente)
    
    config.fechas = fechas

    config.listaid=[]

    #Acomodo para generar la lista de ids.     
    #li = (id.split("["))
    #ids = (li[1].split("]"))[0]
    #ids = list(ids.split(","))

    for a in id_cliente:
      a = a.strip()
      b = "COPERNICUS/S2/"+a
      config.listaid.append(b)

    #Acomodo para generar lista de fechas
    #li = (fechas.split("["))
    #ids = (li[1].split("]"))[0]
    #ids = list(ids.split(","))

   # for a in ids:
    #  a = a.strip()
   #   config.fechas.append(b)
    #Asigno las fechas al drowmenu
    selfechas.index = None
    selfechas.options=config.listaid
    
    with out:
     print("Busqueda Terminada")
    

boton0.on_click(buscar_imagenes)

HBox([boton0,out])

In [10]:
from ipywidgets import Checkbox, HBox, VBox
from IPython.display import display, clear_output
import ipyleaflet

#CONSTANTES DEL MAPA
#palette = ['#FFFFFF', '#CE7E45', '#DF923D', '#F1B555', '#FCD163', '#99B718', '#74A901', '#66A000', '#529400', '#3E8601', '#207401', '#056201', '#004C00', '#023B01', '#012E01', '#011D01', '#011301']
#Map.add_colorbar_branca(colors=palette,vmin=0.2,vmax=0.7)
output_widget = widgets.Output(layout={'border': '1px solid black',"font_weight": "5px"})
output_control = ipyleaflet.WidgetControl(widget=output_widget, position='bottomleft')
Map.add_control(output_control)

#selfechas = widgets.Dropdown(
#    options=config.listaid,
#    value=config.listaid[0],
#    description='Seleccione Fecha:',
#    disabled=False,
#)

boton1 = widgets.Button(description='Graficar NDVI',button_style='primary')
boton2 = widgets.Button(description='Graficar',button_style='primary')

out1 = widgets.Output()





def graficar_onclicked(b):
 #Borro lo anterior
 with out:
  clear_output()
 laimagen = ee.Image(selfechas.value)
 escena = laimagen.clip(config.lote)
 escena = escena.normalizedDifference(['B8', 'B4']).rename('NDVI')
 escena = escena.select(["NDVI"])

 params = escena.reduceRegion(
           reducer= ee.Reducer.percentile([10, 90]), 
           geometry= config.lote, 
           scale= 10,
           )

 parametros = params.getInfo()
 min_valor = [parametros['NDVI_p10']]
 max_valor = [parametros['NDVI_p90']]

 vis_params = {
              'min': min_valor,
              'max': max_valor,
              'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301']}

 with out1:
  texto = "Graficando imagen "+str(selfechas.value)+" NDVI min:"+ str(min_valor) +" NDVI max:"+ str(max_valor) 
  print(texto)

 Map.addLayer(escena, vis_params, selfechas.value)

 with output_widget:
  clear_output()
  print(texto)

def graficar_cv_onclicked(b):
 #Borro lo anterior
 with out:
  clear_output()
 laimagen = ee.Image(selfechas.value)
 escena = laimagen.clip(config.lote.buffer(1000))
 

 params = escena.select(["B4","B3","B2"]).reduceRegion(
      reducer= ee.Reducer.percentile([5, 95]), 
      geometry= config.lote.buffer(1000), 
      scale= 10,
      )
 parametros = params.getInfo()
 min_escena = [parametros['B4_p5'], parametros['B3_p5'], parametros['B2_p5']]
 max_escena = [parametros['B4_p95'], parametros['B3_p95'], parametros['B2_p95']]

 vis_params = {
              'bands':['B4', 'B3', 'B2'],
              'min': min_escena,
              'max': max_escena,
               }

 with out1:
  texto = "Graficando imagen "+str(selfechas.value) 
  print(texto)

 Map.addLayer(escena, vis_params, selfechas.value)

 with output_widget:
  clear_output()
  print(texto)
  

boton1.on_click(graficar_onclicked)
boton2.on_click(graficar_cv_onclicked)

HBox([selfechas,boton1,boton2,out1])

In [7]:
from ipywidgets import Checkbox, HBox, VBox
from IPython.display import display, clear_output
import ipyleaflet

boton2 = widgets.Button(description='Calcular TC',button_style='primary')
out2 = widgets.Output()


def calcular_tc(b):
    with out2:
        print("Iniciando calculo")
    import config
    laimagen = ee.Image(selfechas.value)
    escena = laimagen.clip(config.lote)
    escena = escena.normalizedDifference(['B8', 'B4']).rename('NDVI')
    
    def addArea(feature):
     feature = feature.set({"areaHa": feature.geometry().area().divide(100 * 100)});
     return feature

    config.fet = config.fet.map(addArea);


    # Agregar la media de cada imagen
    resumen = escena.select("NDVI").reduceRegions(config.fet, ee.Reducer.mean())

    medias = resumen.aggregate_array("mean")
    indexsat = resumen.aggregate_array("system:index")
    cultivos = resumen.aggregate_array("CULTIVOS")
    areaHa = resumen.aggregate_array("areaHa")


    test_dict = ee.Dictionary.fromLists(['indice', 'medias',"CULTIVOS","areaHa"], [indexsat, medias,cultivos,areaHa])
    featureCollection = ee.FeatureCollection([ee.Feature(None, test_dict)])
    
    with out2:
        print("Consultando datos del servidor")

    link = featureCollection.getDownloadURL(filetype="CSV", selectors=None, filename=None)
    #Generamos el diccionario con los valores.
    import csv, urllib.request
    response = urllib.request.urlopen(link)
    lines = [l.decode('utf-8') for l in response.readlines()]
    reader = csv.DictReader(lines)
    
    data = list(reader)
    # Converting string to list
    indice_cliente = data[0]["indice"].strip('][').split(', ')
    medias_cliente = data[0]["medias"].strip('][').split(', ')
    cultivo_cliente = data[0]["CULTIVOS"].strip('][').split(', ')
    areaHa_cliente = data[0]["areaHa"].strip('][').split(', ')

    
    areaHa_cliente = map(float, areaHa_cliente)
    areaHa_cliente = list(areaHa_cliente)
    
    medias_cliente = map(float, medias_cliente)
    medias_cliente = list(medias_cliente)

    indice_cliente = map(int, indice_cliente)
    indice_cliente = list(indice_cliente)

    import pandas as pd
    ndvis = pd.DataFrame(list(zip(indice_cliente, medias_cliente,cultivo_cliente,areaHa_cliente)),columns =['Indice', 'MediaNDVI', 'CULTIVOS',"areaHa"])
    
    config.ndvis = ndvis
    
    with out2:
        print("Datos obtenidos")

    #EN ESTA CELDA SE CALCULA LA RADIACION POR FECHA LATITUD Y LONGITUD
    #Cargo los datos de radiacion
    import pandas as pd
    import os
    radiacion = pd.read_csv(os.getcwd()+"\\Radiación incidente2.csv",sep=";")

    #Extraigo coordendas del shapefile cargado
    with out2:
        print("Consultando RF")


    x = config.df1.centroid.x
    y = config.df1.centroid.y

    import numpy as np
    latitudes = np.arange(-48, -18, 0.2)
    longitudes = np.arange(-68, -52, 0.4)

    def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return array[idx]

    valor_lat = (find_nearest(latitudes, y.values[0]))
    valor_lat = round(valor_lat, 1)

    valor_long = (find_nearest(longitudes, x.values[0]))
    valor_long = round(valor_long, 1)

    #Aca tendria que conseguir las fechas de la imagen selecionada
    indice = config.listaid.index(selfechas.value)
    fecha = config.fechas[indice]

    mes = int(fecha.split('-')[1])
    dia = int(fecha.split('-')[2])


    radiacion_filtrada = radiacion[radiacion['MES'] == mes]
    radiacion_filtrada = radiacion_filtrada.iloc[(radiacion_filtrada['DIA']-dia).abs().argsort()[:76]]
    radiacion_filtrada = radiacion_filtrada.iloc[(radiacion_filtrada['lat']-y.values[0]).abs().argsort()[:1]]
    radiacion_filtrada.iloc[(radiacion_filtrada['lat']-y.values[0]).abs().argsort()[:1]]
    radiacion_filtrada = radiacion_filtrada.filter(items=[str(valor_long)])
    nivel_radiacion = radiacion_filtrada.iloc[0][str(valor_long)]

    #EN ESTA CELDA SE CALCULA LA TC
    #Cargo los datos de radiacion
    with out2:
        print("Calculando TC")

    import pandas as pd
    import os
    regresora = pd.read_csv(os.getcwd()+"\\regresoras.csv",sep=";")

    TCarray=[]
    RFfinalarray = []

    for i in range(0,len(ndvis)):
        mediandvizona= ndvis["MediaNDVI"][i]
        cultivozona = ndvis["CULTIVOS"][i]


        #Selecciono el cultivo
        var_cultivo = regresora[regresora['cultivo'] == cultivozona]
        pendiente = var_cultivo.iloc[0]["pendiente"]
        ordenada =  var_cultivo.iloc[0]["origen"]

        a = 10.07
        b = 0.15392254


        def calculotc(media_ndvi,RF,pendiente,ordenada):
            Rfai = RF*0.48
            frFAA = (((1+media_ndvi)/(1-media_ndvi))/a)-b
            if frFAA > 0.95:
             frfaa = 0.95
            RFAA = Rfai*frFAA
            TC = ((RFAA*pendiente)+ordenada)*10

            TCarray.append(TC)
            RFfinalarray.append(RF)
            return 

        calculotc(mediandvizona,nivel_radiacion,pendiente,ordenada)


    ndvis["TC"] = TCarray
    ndvis["RF"] = RFfinalarray
    with out2:
        print("Armando shpefile")
        
    rutafinal = os.getcwd()+"\\"+"salida1.geojson"
    df = gpd.read_file(rutafinal)
    df["MediaNDVI"] = ndvis["MediaNDVI"]
    df["TC"] = ndvis["TC"]
    df["areaHa"] = ndvis["areaHa"]
    
    ruta = "Resultados\\"+fc1.selected_filename
    df.to_file(ruta)
    
    with out2:
        clear_output()
        print("El shape fue gurdado en: "+ruta)
        print(ndvis)
    
    return 


boton2.on_click(calcular_tc)
HBox([boton2,out2])

<ipython-input-7-8fe0e2ff23f8>:82: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x = config.df1.centroid.x
<ipython-input-7-8fe0e2ff23f8>:83: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y = config.df1.centroid.y
